In [1]:
import glob
import pydicom
import nibabel
import os
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd

from tqdm import tqdm

# path to the original NIFTI dataset
path = '../../../Processed NIFTI Dataset/'

# path = '../../../Simple Path Dataset/manifest-1654812109500/Duke-Breast-Cancer-MRI/'

# path to which we want to save the PNG images
img_save_path = '../../../2D Pretrain/'

from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=True,nb_workers= 12)
from pathlib import Path

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
# get all files that aren't the segmentation masks 
files = glob.glob(path + '*/[!segmentation]*.img.gz', recursive=True)

# files = glob.glob(path + '*/*/*/*.dcm', recursive=True)

os.makedirs(img_save_path, mode = 511, exist_ok=True)

In [3]:
Path('../../../Processed NIFTI Dataset\\Breast_MRI_001\\post_1.img.gz').name

'post_1.img.gz'

In [4]:
filepaths = pd.DataFrame.from_records([files]).T.rename(columns = {0: 'filepath'})
filepaths['patient'] = filepaths['filepath'].apply(lambda x: Path(x).parent.name)
filepaths['ser_type'] = filepaths['filepath'].apply(lambda x: Path(x).name.split('.')[0])
filepaths

,filepath,patient,ser_type
0,../../../Processed NIFTI Dataset\Breast_MRI_00...,Breast_MRI_001,post_1
1,../../../Processed NIFTI Dataset\Breast_MRI_00...,Breast_MRI_001,pre
2,../../../Processed NIFTI Dataset\Breast_MRI_00...,Breast_MRI_002,post_1
3,../../../Processed NIFTI Dataset\Breast_MRI_00...,Breast_MRI_002,pre
4,../../../Processed NIFTI Dataset\Breast_MRI_00...,Breast_MRI_003,post_1
...,...,...,...
1839,../../../Processed NIFTI Dataset\Breast_MRI_92...,Breast_MRI_920,pre
1840,../../../Processed NIFTI Dataset\Breast_MRI_92...,Breast_MRI_921,post_1
1841,../../../Processed NIFTI Dataset\Breast_MRI_92...,Breast_MRI_921,pre
1842,../../../Processed NIFTI Dataset\Breast_MRI_92...,Breast_MRI_922,post_1


In [5]:
def process_row(row):
    import os, nibabel
    from PIL import Image
    
    img_save_path = '../../../2D Pretrain/'
    
    
    filename = row['filepath']
    patient = row['patient']
    scan_type = row['ser_type']
    # get patient name and filename
    new_path = os.path.join(img_save_path, patient)
    os.makedirs(new_path, mode = 511, exist_ok=True)
    
    # load data
    scan = nibabel.load(filename)
    scan = scan.get_fdata()
    for plane in range(scan.shape[2]):
        # get relevant subsection
        p = scan[:, :, plane]#.astype(np.uint16)\
        normalized_slice = 255 * (p - p.min()) / (p.max() - p.min())
        normalized_slice = normalized_slice.astype('uint8')
        # create image
        img = Image.fromarray(normalized_slice)
        # note save path
        save_path = f"{new_path}/{scan_type}_{plane}.png"
        
        img.save(save_path)
        
filepaths.parallel_apply(process_row, axis = 1)